# CIFAR10 教學筆記本
在這本筆記本中，我們將利用CIFAR10資料集和深度類神經網路AlexNet，訓練一個影像分類器。

## 程式語言及相關的Library
* Python 
* Numpy
* Pytorch

# 1. 導入相關的Library

In [1]:
import os
import torch
import torchvision
from torch.autograd import Variable
import numpy as np

main_path = os.getcwd()
resizer = torch.nn.functional.interpolate

# 2. 導入CIFAR10 資料集
簡介：Cifar-10是由Geoffrey Hinton教授所整理的影像資料集，在其中包含60000筆解析度為32*32之彩色RGB圖片，其中 50000筆為訓練資料，10000筆為測試資料，此資料庫為深度學習中常用的資料集。
![Cifar 10](./utils/notebook_pic/cifar10.png)
Cifar-10一共包含10種類別的圖片，包含
* 1 : airplain
* 2 : automobile
* 3 : bird
* 4 : cat
* 5 : deer
* 6 : dog
* 7 : frog
* 8 : horse
* 9 : ship
* 10 : truck

In [2]:
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,shuffle=True, num_workers=2) #
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# 3. 定義AlexNet
AlexNet在2012年的ImageNet LSVRC的比賽上拿到了冠軍，使得Convolutional Neural Network開始被學界、業界重視。在近幾年的中，深度學習的論文、應用更是被廣泛的推廣，以開山始祖AlexNet作為認識深度學習的模型是再恰當不過的。

![AlexNet](./utils/notebook_pic/AlexNet.png)

## AlexNet結構及意義
* 五層卷積層 : 掃描圖像上的特徵，並將與結果之正相關特徵激活。
* Maxpooling : 減輕從卷積層掃描之特徵。
* 全連接層: 藉由萃取後的圖像特徵，進行分類。

## AlexNet 特性
* 採用Dropout：避免過擬和訓練資料。
* 採用Relu非線性激活函數：防止梯度消散、加快訓練、增加訓練穩定度。
* 支援GPU訓練。

In [3]:
import torch.nn as nn
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    
Net = AlexNet(num_classes=10)
Net.train()
Net.cuda()
print("AlexNet結構：")
print(Net)

AlexNet結構：
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3)

# 4. 設定訓練細節
* 訓練次數 : 預設為5
* 設定Loss function : 預設為Cross-Entropy Loss
* 設定Optimizer : 預設為SGD
 


In [4]:
import torch.optim as optim
EPOCH = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)

# 5. 進行訓練

In [5]:
for epoch in range(EPOCH): 
    print('Epoch [', epoch,' / ',EPOCH,'] : ')
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 取得訓練資料
        inputs, labels = data
        inputs = Variable(resizer(inputs,size = (227,227))).cuda()
        labels = Variable(labels).cuda()
        # 初始化optimizer
        optimizer.zero_grad()
        # 資料送至AlexNet : Forward
        outputs = Net(inputs)
        # 計算Loss
        loss = criterion(outputs, labels)
        # 修正網路並調整Optimizer
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (i - 1998, i + 1, running_loss / 2000))
            running_loss = 0.0

print('結束訓練')

Epoch [ 0  /  5 ] : 


OSError: [Errno 12] Cannot allocate memory


# 訓練20 EPOCH 的細節
. <br />
. <br />
. <br />
. <br />
Epoch [ 17  /  20 ] : <br />
[1,  2000] loss: 0.111<br />
[2001,  4000] loss: 0.139<br />
Epoch [ 18  /  20 ] : <br />
[1,  2000] loss: 0.107<br />
[2001,  4000] loss: 0.112<br />
Epoch [ 19  /  20 ] : <br />
[1,  2000] loss: 0.085 <br />
[2001,  4000] loss: 0.110 <br />

# 6. 測試訓練後的模型

In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
with torch.no_grad():
    
    for data in testloader:
        images, labels = data
        images = Variable(resizer(images,size = (227,227))).cuda()
        
        outputs = Net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted.data.cpu() == labels).squeeze()
        
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 84 %
Accuracy of   car : 90 %
Accuracy of  bird : 79 %
Accuracy of   cat : 66 %
Accuracy of  deer : 80 %
Accuracy of   dog : 72 %
Accuracy of  frog : 84 %
Accuracy of horse : 87 %
Accuracy of  ship : 90 %
Accuracy of truck : 81 %


# 訓練20 EPOCH 的成果
Accuracy of plane : 84 %<br />
Accuracy of   car : 90 %<br />
Accuracy of  bird : 79 %<br />
Accuracy of   cat : 66 %<br />
Accuracy of  deer : 80 %<br />
Accuracy of   dog : 72 %<br />
Accuracy of  frog : 84 %<br />
Accuracy of horse : 87 %<br />
Accuracy of  ship : 90 %<br />
Accuracy of truck : 81 %<br />